In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 23 20:22:42 2024

@author: 14421
"""
import torch # 张量计算
import torch.nn as nn # 神经网络模块
import torch.optim as optim # 优化器
import pandas as pd # 用于数据处理
from sklearn.model_selection import train_test_split # 用于划分训练集和测试集
from sklearn.preprocessing import StandardScaler, LabelEncoder # 用于标准化和标签编码
import matplotlib.pyplot as plt # 用于绘制损失曲线

# 加载数据
data = pd.read_csv(r"C:\Users\14421\Desktop\银行客户流失\代码及数据\Churn_Modelling.csv")

# 数据预处理
# 选择特征和目标变量
X = data.drop(labels=['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1) # 去除无关的列
y = data['Exited'] # 目标变量：客户是否流失（0：没流失，1：流失）

# 对分类变量进行编码
label_encoder = LabelEncoder()
X['Gender'] = label_encoder.fit_transform(X['Gender']) # 对'Gender'列进行编码
X = pd.get_dummies(X, columns=['Geography'], drop_first=True) # 对'Geography'列进行独热编码

# 特征标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 将数据集拆分为训练集和测试集（80%训练，20%测试）
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 将数据转换为Torch张量
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)
class ChurnPredictor(nn.Module):  # 定义神经网络模型
    def __init__(self):  # 初始化函数
        super(ChurnPredictor, self).__init__()  # 调用父类初始化函数
        self.fc1 = nn.Linear(x_train.shape[1], out_features=64)  # 输入特征维度为x_train的列数，输出维度为64
        self.fc2 = nn.Linear(in_features=64, out_features=32)  # 第二个全连接层
        self.fc3 = nn.Linear(in_features=32, out_features=1)  # 输出层：1个神经元，表示是否流失
        self.relu = nn.ReLU()  # 激活函数：ReLU
        self.sigmoid = nn.Sigmoid()  # 输出层使用Sigmoid激活函数进行二分类
    
    def forward(self, x):  # 前向传播函数
        x = self.relu(self.fc1(x))  # 第一层全连接并应用ReLU激活
        x = self.relu(self.fc2(x))  # 第二层全连接并应用ReLU激活
        x = self.sigmoid(self.fc3(x))  # 输出层经过Sigmoid激活
        return x  # 返回结果

# 实例化模型、定义损失函数和优化器
model = ChurnPredictor()  # 创建模型实例
criterion = nn.BCELoss()  # 二分类交叉熵损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam优化器，学习率为0.001

# 训练模型
num_epochs = 100  # 设置训练轮数为100
for epoch in range(num_epochs):  # 循环遍历每个epoch
    model.train()  # 设置模型为训练模式
    output = model(x_train).squeeze()  # 获取预测结果并压缩为一维
    loss = criterion(output, y_train)  # 计算损失
    optimizer.zero_grad()  # 清空梯度
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    if (epoch + 1) % 10 == 0:  # 每10个epoch打印一次信息
        print(f'Epoch [{(epoch + 1)/num_epochs}], Loss: {loss.item():.4f}')
# 测试模型
model.eval()  # 设置模型为评估模式
with torch.no_grad():  # 禁用梯度计算，以节省内存
    correct = 0
    total = 0
    output = model(x_test).squeeze()  # 获取模型输出
    predicted = (output > 0.5).float()  # 使用0.5作为阈值，二分类
    correct = (predicted == y_test).sum().item()  # 计算正确预测的样本数
    total = y_test.size(0)  # 测试集总样本数
    # 计算准确率
    accuracy = 100 * correct / total  # 计算准确率
    print(f'Test Accuracy: {accuracy:.2f}%')

# 可视化训练损失
# 记录损失值
losses = []

# 重新训练并记录损失
for epoch in range(num_epochs):
    model.train()
    output = model(x_train).squeeze()
    loss = criterion(output, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    
# 绘制损失曲线
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()
